In [1]:
# from src.crawler import Crawler

In [2]:
CRWALING = True         # 크롤링
DB_BULK_INSERT = True   # 크롤링 데이터 DB INSERT

TEST_SIZE = 1           # 크롤링 공고 테스트용 사이트 설정되어 있는 만큼 진행됨 기본값 -1 = 전체 진행
batch_id = ''           # 중간 진입위한 BATCH_ID 넣으면 

# 중간에 끊긴 경우: 아래 주석 해제하고 실행하면 마지막 batch_id로 이어서 진행
# from src.database import DataBaseHandler
# with DataBaseHandler() as db:
#     result = db.execute_query("SELECT batch_id FROM annc_lh_temp ORDER BY batch_start_dttm DESC LIMIT 1")
#     if result:
#         batch_id = str(result[0]['batch_id'])
#         print(f"마지막 batch_id: {batch_id}")

if batch_id:
    CRWALING = False
    DB_BULK_INSERT = False

In [3]:
import os
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

True

## 1. 테이블 존재 여부 확인

## 0. DB 초기화 (선택사항)
기존 데이터를 모두 삭제하고 테이블을 재생성합니다.  
**주의: 모든 공고, 파일, 청크 데이터가 삭제됩니다!**

In [5]:
from src.database import DataBaseHandler

tables = ("annc_all","annc_files","annc_lh_temp","doc_chunks")

cases = ','.join([f"""sum(case when table_name in ('{table}') then 1 else 0 end) as {table}""" for table in tables])

query = f"""
SELECT 
{cases}
FROM information_schema.tables
WHERE table_schema = 'public'
AND table_name = any(%s)
"""

with DataBaseHandler() as db:
    result = db.execute_query(query, (list(tables),))

zero_count = 0
for val in result[0].items():
    table_name = val[0]
    table_count = val[1]
    print(f"{val[0]}: {'✅' if val[1] > 0 else '❌'}")
    if table_count == 0:
        zero_count += 1
if zero_count > 0:
    print("테이블 이상 상태 확인 필요")
    raise

annc_all: ✅
annc_files: ✅
annc_lh_temp: ✅
doc_chunks: ✅


## 2. 크롤링

In [6]:
# 크롤링 본 로직

from src.crawler.lh import LH


lh_crwaler = LH()

if CRWALING:
    df_all_annc = lh_crwaler.crawl_lh_notices_all_data('접수중')
    df_all_annc += lh_crwaler.crawl_lh_notices_all_data('공고중')
    # df_all_annc += lh_crwaler.crawl_lh_notices_all_data('정정공고중')
else:
    df_all_annc = []
    print("크롤링 x")

-----------------------------------------------------------------------------
🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...
**필터 기준: [유형 '접수중'] + 게시일 2024-11-01 이후 데이터 수집.**

📄 Crawling page 1 (0 to 1000)...
✅ 페이지 1 로드 성공. 12개 중 12개 데이터 추출.
🏁 크롤링 종료 조건 충족. 전체 크롤링 종료.

✨ 전체 크롤링 완료. 총 12건의 데이터를 수집했습니다.
-----------------------------------------------------------------------------
🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...
**필터 기준: [유형 '공고중'] + 게시일 2024-11-01 이후 데이터 수집.**

📄 Crawling page 1 (0 to 1000)...
✅ 페이지 1 로드 성공. 131개 중 131개 데이터 추출.
🏁 크롤링 종료 조건 충족. 전체 크롤링 종료.

✨ 전체 크롤링 완료. 총 131건의 데이터를 수집했습니다.


In [7]:
# 크롤링 된 데이터 중복제거

len_before = len(df_all_annc)
unique_tuples = set(tuple(sorted(d.items())) for d in df_all_annc)
df_all_annc = [dict(t) for t in unique_tuples]
len_after = len(df_all_annc)

print(f'중복제거 {len_before} ➡️ {len_after}')

중복제거 143 ➡️ 131


In [8]:
len_before = len(df_all_annc)
df_all_annc = [annc for annc in df_all_annc if annc['annc_type'] in ('매입','임대')]
len_after = len(df_all_annc)

print(f'기타항목 제거 {len_before} ➡️ {len_after}')

기타항목 제거 131 ➡️ 38


In [9]:
len_before = len(df_all_annc)
# df_all_annc = [annc for annc in df_all_annc if annc['annc_region'] in ('서울특별시','경기도')]
df_all_annc = [annc for annc in df_all_annc]
len_after = len(df_all_annc)

print(f'전지역 {len_before} ➡️ {len_after}')

전지역 38 ➡️ 38


In [10]:
df_all_annc

[{'annc_deadline_dt': '2025.12.31',
  'annc_dtl_type': '행복주택',
  'annc_pblsh_dt': '2025.05.28',
  'annc_region': '경기도',
  'annc_status': '공고중',
  'annc_title': '양주시, 동두천시 행복주택 상시모집[선착순동호지정, 입주자격완화, 선계약후검증]',
  'annc_type': '임대',
  'annc_url': 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300018161&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=10',
  'lh_ais_tp_cd': '10',
  'lh_ccr_cnnt_sys_ds_cd': '03',
  'lh_ls_sst': '',
  'lh_pan_id': '2015122300018161',
  'lh_upp_ais_tp_cd': '06'},
 {'annc_deadline_dt': '2025.12.12',
  'annc_dtl_type': '매입임대',
  'annc_pblsh_dt': '2025.11.27',
  'annc_region': '광주광역시 외',
  'annc_status': '접수중',
  'annc_title': '광주전남 기존주택 등 매입임대주택 예비입주자 모집 공고',
  'annc_type': '임대',
  'annc_url': 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300019098&ccrCnntSysDsCd=03&uppAisTpCd=13&aisTpCd=26',
  'lh_ais_tp_cd': '26',
  'lh_ccr_cnnt_sys_ds_cd': '03',
  'lh_ls_sst': '',
  'lh_pan_id': '2015

# 3. 대상 DB INSERT - BATCH ID 발행

In [11]:
from src.database.repository import (
    AnncLhRepository, AnncQrRepository,
    AnncAllRepository, AnncFileRepository,
    DocChunkRepository,
)

lh_repo = AnncLhRepository()
# qr_repo = AnncQrRepository()
all_repo = AnncAllRepository()
file_repo = AnncFileRepository()
dc_repo = DocChunkRepository()

if DB_BULK_INSERT: 
    batch_id = lh_repo.bulk_insert_announcements(df_all_annc)
else:
    print("크롤링 데이터 삽입 x")

if not batch_id:
    raise "batch_id가 없으면 진행 불가."


In [12]:
batch_id

'f97e6547-cc22-4508-8140-dd662fc82661'

In [13]:
annc_list_lh = AnncQrRepository().get_announcements_merge_target(batch_id, annc_status=['공고중','접수중'])

print(f'{len(annc_list_lh)}건 조회 완료')

31건 조회 완료


# 4. 파일 등록

In [14]:
from datetime import datetime, timedelta
from src.parser import parse_pdf
from src.chunker import create_chunks_from_elements
from src.embedder import embed_chunks
import json

def process(row_lh, corp_cd):
    """
    공고 리스트
    """
    time_laps = []

    title_now = lambda title: (title, datetime.now())

    # 1. 배치 테이블 상태 변경 -> 시작
    lh_repo.update_announcements('START', row_lh['batch_id'], row_lh['batch_seq'])
    time_laps.append(title_now(f"배치 시작 - {row_lh['annc_title']}"))

    # 2. 공고 닫기 처리
    row_lh['corp_cd'] = corp_cd
    row_lh['service_status'] = 'CLOSE'
    row_lh['created_at'] = datetime.now()
    row_lh['updated_at'] = datetime.now()

    print(row_lh)
    merge_result = all_repo.merge_announcements([row_lh,]) # 원래 다건을 위한것
    time_laps.append(title_now(f"공고 닫기 처리"))

    if not merge_result:
        raise Exception("머지된 행 없음")
    
    merge_result = merge_result[0]
    annc_id = merge_result['annc_id']

    # 3. 파일 조회
    file_list = lh_crwaler.get_file_list(row_lh)
    time_laps.append(title_now(f"파일 조회"))

    if not file_list:
        raise Exception("파일 없음")
    
    # 4. 파일, 청크 벡터 초기화
    dc_repo.delete_chunks_by_annc_id(annc_id)
    file_repo.delete_files_by_annc_id(annc_id)
    # print(f"{get_elapsed_time()} {batch_msg} - 파일 초기화")
    time_laps.append(title_now(f"파일 기록 삭제 처리"))

    for idx_file, file_info in enumerate(file_list):
        time_laps.append(title_now(f"파일 처리 시작 ({idx_file+1}/{len(file_list)})"))

        annc_file = {}

        annc_file['annc_id'] = annc_id
        annc_file['file_name'] = file_info['cmnAhflNm']
        annc_file['file_type'] = file_info['slPanAhflDsCdNm']
        annc_file['file_ext'] = 'pdf'

        # 파일 다운
        file_path, annc_file = lh_crwaler.down_file(file_info['cmnAhflSn'], annc_file)
        time_laps.append(title_now(f"파일 다운로드"))

        # 파일 DB 등록
        inserted_file_info = file_repo.bulk_insert_files([annc_file])[0]
        file_id, file_name = inserted_file_info['file_id'], inserted_file_info['file_name']
        time_laps.append(title_now(f"파일 정보 DB 기록"))

        # 파일 엘리먼트 구성
        parsed = parse_pdf(file_path, annc_id)
        time_laps.append(title_now(f"파일 -> Markdown"))
        # test_elements = parsed.elements[:10]

        chunks = create_chunks_from_elements(parsed.elements, annc_id)
        time_laps.append(title_now(f"Markdown -> 청크"))

        embed_chunks(chunks)
        time_laps.append(title_now(f"청크 -> 임베딩"))

        chunk_dto = [{
            'file_id': file_id,
            'annc_id': annc_id,
            'chunk_type': c.element_type,#get('element_type','text'),
            'chunk_text': c.text, #get('text',''),
            'page_num': c.page_number,
            'embedding': c.embedding,
            'metadata': json.dumps(c.metadata)  # dict를 JSON 문자열로 변환
        } for c in chunks]

        
        dc_repo.bulk_insert_chunks(chunk_dto)
        time_laps.append(title_now(f"임베딩 정보 기록"))

        # 사용한 파일 삭제
        if os.path.exists(file_path):
            os.remove(file_path)
        time_laps.append(title_now(f"파일 삭제"))

        # return chunks
    lh_repo.update_announcements('COMPLETE', row_lh['batch_id'], row_lh['batch_seq'])
    time_laps.append(title_now(f"배치 종료"))

    # print(parsed)
    row_lh['service_status'] = 'OPEN'
    merge_result = all_repo.merge_announcements([row_lh,]) # 원래 다건을 위한것
    time_laps.append(title_now(f"배치 완료 처리"))

    return time_laps

In [15]:
def print_time_laps(time_laps):
    """
    process()의 반환값(time_laps)을 받아
    - 직전 단계 대비 소요시간
    - 시작 이후 총 소요시간
    을 함께 출력한다.
    """
    if not time_laps:
        print("시간 정보가 없습니다.")
        return

    start_time = time_laps[0][1]

    for i, (title, t) in enumerate(time_laps):
        if i == 0:
            diff_prev = timedelta(0)
        else:
            diff_prev = t - time_laps[i-1][1]

        diff_total = t - start_time

        print(f"[{i+1:02d}] {title}")
        print(f"  ├ 전 단계 대비: {diff_prev}")
        print(f"  └ 전체 경과   : {diff_total}")

## 실행 및 타임 랩스

In [16]:
for row in annc_list_lh:
    time_laps = process(row,"LH")

    print_time_laps(time_laps)


{'batch_id': 'f97e6547-cc22-4508-8140-dd662fc82661', 'batch_seq': 1, 'annc_title': '양주시, 동두천시 행복주택 상시모집[선착순동호지정, 입주자격완화, 선계약후검증]', 'annc_url': 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300018161&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=10', 'batch_status': 'PENDING', 'batch_start_dttm': datetime.datetime(2025, 12, 11, 7, 19, 6, 557176, tzinfo=datetime.timezone.utc), 'annc_type': '임대', 'annc_dtl_type': '행복주택', 'annc_region': '경기도', 'annc_pblsh_dt': '2025.05.28', 'annc_deadline_dt': '2025.12.31', 'annc_status': '공고중', 'lh_pan_id': '2015122300018161', 'lh_ais_tp_cd': '10', 'lh_upp_ais_tp_cd': '06', 'lh_ccr_cnnt_sys_ds_cd': '03', 'lh_ls_sst': '', 'corp_cd': 'LH', 'service_status': 'CLOSE', 'created_at': datetime.datetime(2025, 12, 11, 16, 19, 11, 116661), 'updated_at': datetime.datetime(2025, 12, 11, 16, 19, 11, 116661)}
[('양주시, 동두천시 행복주택 상시모집[선착순동호지정, 입주자격완화, 선계약후검증]', 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026

c:\Users\Playdata\anaconda3\envs\zf2_back\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id 9d8c8f40-d307-4910-a5f7-a41db46e1f3e
[('양주시, 동두천시 행복주택 상시모집[선착순동호지정, 입주자격완화, 선계약후검증]', 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300018161&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=10', datetime.datetime(2025, 12, 11, 16, 19, 11, 116661), datetime.datetime(2025, 12, 11, 16, 19, 11, 116661), 'LH', '임대', '행복주택', '경기도', '2025.05.28', '2025.12.31', '공고중', 'OPEN')]
[01] 배치 시작 - 양주시, 동두천시 행복주택 상시모집[선착순동호지정, 입주자격완화, 선계약후검증]
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.045702
  └ 전체 경과   : 0:00:00.045702
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.179837
  └ 전체 경과   : 0:00:00.225539
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.093102
  └ 전체 경과   : 0:00:00.318641
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00.318641
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.424412
  └ 전체 경과   : 0:00:00.743053
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.069447
  └ 전체 경과   : 0:00:00.812500
[08] 파일 -> Markdown
  ├ 전 

c:\Users\Playdata\anaconda3\envs\zf2_back\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id 18740723-172d-4d15-b0bc-492147058a7a


KeyboardInterrupt: 